In [ ]:
######################################################
'''  ### Installation Process, no Conda environment### 

C:\xxx>py -3.6-64 -m pip install scikit-learn numpy pandas
Collecting scikit-learn
...
....
.....

Successfully installed joblib-1.0.1 numpy-1.19.5 pandas-1.1.5 python-dateutil-2.8.1 pytz-2021.1 scikit-learn-0.24.1 scipy-1.5.4 six-1.15.0 threadpoolctl-2.1.0
You are using pip version 18.1, however version 21.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.

C:\xxx>py -3.6-64 -m pip install onnxruntime
Collecting onnxruntime
...
....
.....

Installing collected packages: protobuf, onnxruntime
  The script onnxruntime_test.exe is installed in 'C:\xxx\Python\Python36\Scripts' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Successfully installed onnxruntime-1.6.0 protobuf-3.15.3
You are using pip version 18.1, however version 21.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
'''

######################################################
'''  ### onnxruntime example ###
import onnxruntime as rt
import numpy as np

# Load scikit-learn model and do prediction

def get_prediction(model_path, X):
    # convert list to numpy array
    X = np.array(X)
    # "sees.run" expect 2D array input.

    sess = rt.InferenceSession(model_path)

    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name

    # Do prediction
    pred_onx = sess.run([label_name], {input_name: X.astype(np.float32)})[0]
    
    # int64 would be returned
    return pred_onx
'''
######################################################

In [ ]:
# How to install sklearn-onnx
# https://github.com/onnx/sklearn-onnx
# I tried "conda instal skl2onnx" but conda couldn't find such package.
# "pip install skl2onnx" successfully installed.

# Tutorial
# http://onnx.ai/sklearn-onnx/introduction.html

import numpy as np
import pandas as pd
import seaborn as sns
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
pd.options.display.float_format = '{:.4}'.format

In [ ]:
# dataset musn't have null values

dataset = pd.read_csv('dataset.csv',header=None)
print(f'Total columns include null: {dataset.isnull().sum().sum()}')

In [ ]:
X = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1].astype(int)

from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

print(y_te)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

classifier.fit(X_tr,y_tr)
y_pred = classifier.predict(X_te)

from sklearn.metrics import confusion_matrix

cn = confusion_matrix(y_te, y_pred)
print(cn)

# Convert sklearn model into ONNX format
initial_type = [('float_input', FloatTensorType([None, len(X.columns)]))]
onx = convert_sklearn(classifier, initial_types=initial_type)
with open("my_classifier.onnx", "wb") as f:
    f.write(onx.SerializeToString())